In [1]:
import numpy as np

from astroquery.skyview import SkyView

import urllib.request
import os.path as path

from astropy.io import fits
from astropy.stats import sigma_clipped_stats

import glob

from PIL import Image

In [2]:
with open('mirabestdata.txt', 'r') as f:
    data = f.read().splitlines()
    
dataset = []

# Splitting out the relevant columns: in order, RA, dec, z, size_rad and FR_class
for i in range (1, len(data)):
    
    columns = data[i].split()
    
    # A filter to remove any with radial extent greater than the image size
    if float(columns[7]) < 270:
    
    # A filter to remove any class 400 or 402 objects; these are "unclassifiable" and useless for training
        if columns[8] != '400' and columns[8] != '402':
    
            if i == 1:
    
                dataset = (np.asarray(columns[3:6] + columns[7:9])).astype(np.float)
        
            else:
        
                columns = (np.asarray(columns[3:6] + columns[7:9])).astype(np.float)
                dataset = np.concatenate((dataset, columns))

# Final dataset with arrays of data for individual objects
dataset = np.reshape(dataset, (-1, 5))

In [3]:
# Function to generate the appropriate filename convention for any entry    
def name_string(entry):
    """This takes an entry with columns RA, dec, z, size_rad and class and makes a string to label it"""
    
    label = entry[4].astype(int).astype(str)
    ra = '{:07.3f}'.format(entry[0]*15)  # Moving both into degrees
    dec = '{:+08.3f}'.format(entry[1])  # Retaining sign to keep length consistent
    z = '{:06.4f}'.format(entry[2]) # All redshifts are < 0.5, so keep four significant figures
    rad = '{:07.2f}'.format(entry[3]) # Radial size is a maximum of four figures before point
    
    name = label + '_' + ra + dec + '_' + z + '_' + rad
    
    return name

In [4]:
def image_download(entry, extension='_F', survey='VLA FIRST (1.4 GHz)', pixels=300):
    """Download an image from an entry of the same format as previously"""
    
    # Creating the path to the file and the name it'll be saved as
    filename = './MiraBest'+extension+'/FITS/'+name_string(entry)+'.fits'
    
    # Preventing any duplicate downloads
    if path.exists(filename) == False:
    
        coords = (entry[0]*15).astype(str)+', '+entry[1].astype(str)
    
        print(coords)
    
        location=SkyView.get_image_list(position=coords, survey=survey, pixels=pixels)
        
        print(location)
        
        try:
            
            urllib.request.urlretrieve(location[0], filename)
            print(filename)
            
        except:
            
            print('Problem with url', location)

In [5]:
for i in range(len(dataset)):
    
    image_download(dataset[i], extension='_N', survey='NVSS', pixels=300)
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [6]:
# Hongming's image_convert function to create pngs
def image_convert(name, image_data):
    """
       This function writes a PNG file from a numpy array.
       Args:
       name: Name of the output file without the .png suffix
       image_data: Input numpy array
       Returns:
       Writes PNG file to disk.
       Raises:
       KeyError: Raises an exception.
    """
    im = Image.fromarray(image_data)
    im = im.convert('L')
    im.save(name+".png")
    return

In [7]:
# A modified version of Hongming's crop_center function
def crop_centre(img, cropx, cropy):
    """"
       This function crop images from centre to given size.
       Args:
       img: input image
       cropx: output image width
       cropy: output image height
       Returns:
       data of cropped img
       Raises:
    """
    
    xsize = np.shape(img)[0] # image width
    ysize = np.shape(img)[1] # image height
    startx = xsize//2-(cropx//2)
    starty = ysize//2-(cropy//2)
    img_slice = img[starty:starty+cropy,startx:startx+cropx]
    # This is a sub-optimal solution
    return img_slice

In [8]:
# Nicked from alkasm on stackoverflow
def create_circular_mask(h, w, center=None, radius=None):

    if center is None: # use the middle of the image
        center = (np.rint(w/2), np.rint(h/2))
    if radius is None: # use the smallest distance between the center and image walls
        radius = min(center[0], center[1], w-center[0], h-center[1])

    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)

    mask = dist_from_center <= radius
    return mask

In [9]:
import matplotlib.pyplot as plt

testmatrix = np.random.rand(150,150)
print(np.average(testmatrix))

mask=create_circular_mask(150, 150, radius=75)

subset = testmatrix[mask]
testmatrix[~mask] = 0
print(np.average(testmatrix))

image_max, image_min = subset.max(), subset.min()
print(image_max, image_min)
testmatrix[mask] = (testmatrix[mask] - image_min)/(image_max - image_min)

print(np.average(testmatrix))
print(testmatrix)
plt.imshow(testmatrix)
plt.show()

0.5012441580710394
0.3940367787061799
0.9998363664406926 2.1270699714692753e-05
0.39409295019916396
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


<Figure size 640x480 with 1 Axes>

In [19]:
# A modified version of Hongming's read_fits_image function; now creates all required pngs
def read_fits_image(fitsfile, extension='_F', cropsize=150, pixel_arcsec = 1.8, angularcrop=False):
    """
       This function extracts the image data from a FITS image, clips
       and linearly scales it.
       Args:
       fitsfile: Path to the input FITS file
       Returns:
       image_data: Numpy array containing image from FITS file
       Raises:
       KeyError: Raises an exception.
    """
    
    # Obtaining the naming convention
    namestring = fitsfile[18:52]
    
    with fits.open(fitsfile, ignore_missing_end=True) as hdu:
        image_data = hdu[0].data
        hdu.close()
    image_data = fits.getdata(fitsfile)
    
#     # Check if the file has already been saved as a raw png
#     if path.exists('./MiraBest'+extension+'/PNG/Raw/'+namestring+'_raw.png') is False:
    
#         # Save for the first time here, as 'name_raw.png'
#         image_convert('./MiraBest'+extension+'/PNG/Raw/'+namestring+'_raw', image_data)
    
    # set pixels < 3*noise to zero:
    a = sigma_clipped_stats(image_data,sigma=3.0, maxiters=5)
    image_data[np.where(image_data<=3*a[2])] = 0.0
    
#     # Check if the file has already been saved as a clipped png
#     if path.exists('./MiraBest'+extension+'/PNG/Clip/'+namestring+'_clip.png') is False:
    
#         # Save for the second time here, as 'name_clip.png'
#         image_convert('./MiraBest'+extension+'/PNG/Clip/'+namestring+'_clip', image_data)
    
    image_data = crop_centre(image_data, cropsize, cropsize)
    
#     # Check if the file has already been saved as a cropped png
#     if path.exists('./MiraBest'+extension+'/PNG/Crop/'+namestring+'_crop.png') is False:
    
#         # Save for the third time here, as 'name_crop.png'
#         image_convert('./MiraBest'+extension+'/PNG/Crop/'+namestring+'_crop', image_data)
    
    # Collect size data from name string
    source_size = float(namestring[27:40]) # Extent of radio source in arcsec
    pixel_size = np.ceil(source_size/pixel_arcsec) # Converting to a size in pixels, rounded up to nearest pixel
    pixel_radius = np.ceil(pixel_size/2)
    pixel_radius = np.ceil(pixel_radius*1.5)
    
    if pixel_radius > 75:
        
        pixel_radius = 75
    
    if angularcrop == False:
        
        mask = create_circular_mask(cropsize, cropsize, radius = cropsize/2)
        
    elif angularcrop == True:
        
        mask = create_circular_mask(cropsize, cropsize, radius = pixel_radius)
    
    subset = image_data[mask]
    image_data[~mask] = 0
    
    # normalise to [0, 1]:
    image_max, image_min = subset.max(), subset.min()
    image_data[mask] = (image_data[mask] - image_min)/(image_max - image_min)
    # remap to [0, 255] for greyscale:
    image_data*=255.0
    
    # Check if the file has already been saved as the final png
    if path.exists('./MiraBest'+extension+'/PNG/Scaled_Final/'+namestring+'.png') is False:
    
        # Save for a final time here, as 'name.png'
        image_convert('./MiraBest'+extension+'/PNG/Scaled_Final/'+namestring, image_data)
    
    return image_data

In [20]:
list = glob.glob('./MiraBest_F/FITS/*.fits')

# List of files confirmed to have missing data
blacklist = ['100_036.061+000.563_0.1400_0093.07', '200_003.198+000.788_0.1484_0026.82', '200_033.655+000.710_0.2900_0030.76', '210_348.925-000.435_0.0910_0050.37']

for i in range(len(list)):
    
    # Create images for any file not in the blacklist
    if list[i][18:52] not in blacklist:
        
        print(list[i])
        read_fits_image(list[i], extension='_F', cropsize=150, pixel_arcsec = 1.8, angularcrop = True)

./MiraBest_F/FITS/100_136.428+042.354_0.1565_0016.04.fits
./MiraBest_F/FITS/110_183.885+018.714_0.1798_0025.44.fits
./MiraBest_F/FITS/210_175.474+005.435_0.3642_0012.73.fits
./MiraBest_F/FITS/200_211.430+013.970_0.2976_0008.38.fits
./MiraBest_F/FITS/100_196.932+056.851_0.2500_0084.49.fits
./MiraBest_F/FITS/200_228.561+023.452_0.0876_0031.73.fits
./MiraBest_F/FITS/200_174.054+040.043_0.2904_0010.00.fits
./MiraBest_F/FITS/201_159.868+005.604_0.0908_0099.41.fits
./MiraBest_F/FITS/110_137.562+053.664_0.1857_0014.12.fits
./MiraBest_F/FITS/110_192.988+030.157_0.2235_0011.94.fits
./MiraBest_F/FITS/200_230.366+001.956_0.2435_0023.94.fits
./MiraBest_F/FITS/100_222.254+040.012_0.1854_0188.05.fits
./MiraBest_F/FITS/200_196.885+044.057_0.2025_0107.51.fits
./MiraBest_F/FITS/110_124.725+022.796_0.0958_0034.36.fits
./MiraBest_F/FITS/200_120.280+017.980_0.1400_0049.21.fits
./MiraBest_F/FITS/210_339.278-000.042_0.1615_0044.52.fits
./MiraBest_F/FITS/200_170.675+021.558_0.1912_0009.78.fits
./MiraBest_F/F

/Users/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in less_equal


./MiraBest_F/FITS/110_121.506+019.104_0.0979_0038.08.fits
./MiraBest_F/FITS/200_252.239+026.078_0.1373_0020.84.fits
./MiraBest_F/FITS/100_219.702+000.682_0.1503_0054.27.fits
./MiraBest_F/FITS/200_176.620+006.885_0.2330_0011.08.fits
./MiraBest_F/FITS/200_126.488+052.505_0.4667_0050.25.fits
./MiraBest_F/FITS/100_159.159+014.024_0.2007_0020.15.fits
./MiraBest_F/FITS/210_138.584+017.436_0.3284_0022.34.fits
./MiraBest_F/FITS/200_334.678-009.361_0.1864_0061.39.fits
./MiraBest_F/FITS/102_158.066+031.877_0.3342_0018.55.fits
./MiraBest_F/FITS/100_181.244+045.994_0.2130_0072.27.fits
./MiraBest_F/FITS/210_000.676-000.222_0.2989_0035.63.fits
./MiraBest_F/FITS/200_222.172+017.410_0.2523_0031.68.fits
./MiraBest_F/FITS/200_137.935+006.169_0.1756_0072.60.fits
./MiraBest_F/FITS/200_219.706+034.265_0.3407_0010.29.fits
./MiraBest_F/FITS/100_149.318+019.114_0.0904_0131.03.fits
./MiraBest_F/FITS/210_201.546+009.551_0.2483_0013.94.fits
./MiraBest_F/FITS/100_242.912+015.549_0.3459_0113.72.fits
./MiraBest_F/F

./MiraBest_F/FITS/200_154.977+039.506_0.1121_0104.92.fits
./MiraBest_F/FITS/210_206.387+005.770_0.1250_0043.47.fits
./MiraBest_F/FITS/210_215.494+044.997_0.2817_0020.81.fits
./MiraBest_F/FITS/100_141.413+036.452_0.1121_0168.26.fits
./MiraBest_F/FITS/200_161.634+054.600_0.1447_0264.34.fits
./MiraBest_F/FITS/210_228.725+042.013_0.1349_0068.81.fits
./MiraBest_F/FITS/104_227.499+033.463_0.1102_0029.03.fits
./MiraBest_F/FITS/210_136.586+047.869_0.2400_0043.83.fits
./MiraBest_F/FITS/200_137.973+037.404_0.1043_0030.64.fits
./MiraBest_F/FITS/100_237.206+036.176_0.2334_0051.02.fits
./MiraBest_F/FITS/200_175.915+046.356_0.1160_0007.93.fits
./MiraBest_F/FITS/100_210.563+023.301_0.1613_0065.80.fits
./MiraBest_F/FITS/200_161.261+047.300_0.1449_0021.69.fits
./MiraBest_F/FITS/110_219.183+008.125_0.0505_0205.73.fits
./MiraBest_F/FITS/210_131.623+016.423_0.3496_0007.67.fits
./MiraBest_F/FITS/100_116.785+017.291_0.1876_0107.21.fits
./MiraBest_F/FITS/102_037.617+001.147_0.2673_0031.83.fits
./MiraBest_F/F

./MiraBest_F/FITS/100_233.211+057.574_0.1519_0066.98.fits
./MiraBest_F/FITS/100_167.886+016.439_0.2165_0093.36.fits
./MiraBest_F/FITS/102_187.468+011.677_0.0828_0069.12.fits
./MiraBest_F/FITS/210_145.811+012.160_0.2429_0109.30.fits
./MiraBest_F/FITS/200_128.130+035.287_0.2688_0037.85.fits
./MiraBest_F/FITS/200_195.655+016.467_0.4956_0008.27.fits
./MiraBest_F/FITS/100_155.298+018.636_0.1196_0176.33.fits
./MiraBest_F/FITS/100_175.299+005.735_0.0974_0063.84.fits
./MiraBest_F/FITS/100_239.457+016.310_0.0370_0027.70.fits
./MiraBest_F/FITS/100_138.316+052.149_0.1900_0071.77.fits
./MiraBest_F/FITS/110_257.528+061.519_0.1800_0082.20.fits
./MiraBest_F/FITS/200_005.282-000.925_0.1082_0041.40.fits
./MiraBest_F/FITS/100_122.412+034.927_0.0825_0124.74.fits
./MiraBest_F/FITS/100_225.017+013.463_0.1116_0114.75.fits
./MiraBest_F/FITS/210_165.725+012.984_0.1405_0165.99.fits
./MiraBest_F/FITS/100_198.490+032.113_0.1868_0019.97.fits
./MiraBest_F/FITS/200_149.673+060.228_0.2162_0016.00.fits
./MiraBest_F/F

...

/Users/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:69: RuntimeWarning: invalid value encountered in true_divide


./MiraBest_F/FITS/210_135.129+013.752_0.1922_0013.59.fits
./MiraBest_F/FITS/200_205.715+031.181_0.2846_0080.34.fits
./MiraBest_F/FITS/210_144.028+025.905_0.1653_0007.09.fits
./MiraBest_F/FITS/200_228.197-001.740_0.1461_0107.13.fits
./MiraBest_F/FITS/200_195.281-003.448_0.0834_0010.12.fits
./MiraBest_F/FITS/210_247.569+014.586_0.2790_0106.72.fits
./MiraBest_F/FITS/200_249.598+022.661_0.1757_0093.03.fits
./MiraBest_F/FITS/200_241.045+030.107_0.1736_0010.07.fits
./MiraBest_F/FITS/200_206.263+009.790_0.1303_0089.12.fits
./MiraBest_F/FITS/100_172.683+034.913_0.2047_0033.10.fits
./MiraBest_F/FITS/200_247.102+023.019_0.2079_0058.60.fits
./MiraBest_F/FITS/210_182.767+015.782_0.2683_0058.12.fits
./MiraBest_F/FITS/200_336.088-000.522_0.4066_0036.69.fits
./MiraBest_F/FITS/200_145.350+039.745_0.1075_0074.85.fits
./MiraBest_F/FITS/110_116.399+033.963_0.0625_0032.56.fits
./MiraBest_F/FITS/200_205.700+038.206_0.1720_0109.67.fits
./MiraBest_F/FITS/200_164.784+026.571_0.1752_0013.70.fits
./MiraBest_F/F

/Users/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: RuntimeWarning: invalid value encountered in less_equal


./MiraBest_F/FITS/110_222.340+025.557_0.3741_0022.29.fits
./MiraBest_F/FITS/110_186.195+018.426_0.1690_0014.13.fits
./MiraBest_F/FITS/100_160.305+056.809_0.1854_0034.78.fits
./MiraBest_F/FITS/200_237.540+008.144_0.2320_0029.19.fits
./MiraBest_F/FITS/100_242.045+032.905_0.3038_0116.27.fits
./MiraBest_F/FITS/100_221.676+003.935_0.2691_0027.34.fits
./MiraBest_F/FITS/210_225.444+024.654_0.1203_0032.83.fits
./MiraBest_F/FITS/200_157.868+008.723_0.1410_0051.59.fits
./MiraBest_F/FITS/210_193.094+003.265_0.0988_0051.06.fits
./MiraBest_F/FITS/110_200.263+058.549_0.0528_0074.17.fits
./MiraBest_F/FITS/100_175.143+044.097_0.3837_0074.49.fits
./MiraBest_F/FITS/100_167.342+023.919_0.1869_0057.38.fits
./MiraBest_F/FITS/200_242.280+022.631_0.2659_0011.99.fits
./MiraBest_F/FITS/200_218.704-002.250_0.2906_0051.40.fits
./MiraBest_F/FITS/110_132.519+023.214_0.3557_0028.99.fits
./MiraBest_F/FITS/210_252.864+033.142_0.4425_0050.10.fits
./MiraBest_F/FITS/100_231.229+022.443_0.3747_0050.21.fits
./MiraBest_F/F

./MiraBest_F/FITS/200_140.970+014.283_0.1414_0017.69.fits
./MiraBest_F/FITS/100_240.162+020.981_0.1739_0053.88.fits
./MiraBest_F/FITS/100_217.948+060.853_0.1130_0063.04.fits
./MiraBest_F/FITS/210_336.509-001.235_0.4046_0043.88.fits
./MiraBest_F/FITS/100_344.338-001.138_0.1810_0064.07.fits
./MiraBest_F/FITS/110_180.801+014.865_0.2006_0019.60.fits
./MiraBest_F/FITS/102_169.640+031.253_0.2198_0060.00.fits
./MiraBest_F/FITS/200_334.724-009.295_0.1899_0012.05.fits
./MiraBest_F/FITS/200_200.324+042.588_0.0794_0162.77.fits
./MiraBest_F/FITS/210_144.645+045.340_0.4501_0038.17.fits
./MiraBest_F/FITS/102_134.488+017.080_0.3228_0044.82.fits
./MiraBest_F/FITS/100_128.445+045.255_0.1800_0085.36.fits
./MiraBest_F/FITS/100_203.420+004.252_0.1704_0069.15.fits
./MiraBest_F/FITS/200_207.275+018.975_0.1881_0030.64.fits
